Combined with the API reference docs, this notebook gives an overview of `voeventdb.remote`, for those who are already familiar with Python, VOEvents, and the basic ideas around web-based API usage. A gentler introduction can be found in the rest of the tutorial notebooks.

In [ ]:
from __future__ import print_function
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys, OrderValues
from voeventdb.remote.helpers import Synopsis
from datetime import datetime
import pytz

In `voeventdb.remote` endpoints are represented by functions, and can be found under the `apiv1` namespace.

In [ ]:
#Get a count of VOEvents in the database
apiv1.count()

In [ ]:
#List the IVORNs of the 2 VOEvents most recently inserted into the database
apiv1.list_ivorn(order=OrderValues.id_desc,
                 n_max=2)

We can apply filters to all endpoints except those returning a single packet. If no filters are passed then the result-set is effectively the entire database. 

Filter-sets are defined as dictionaries:

In [ ]:
#Filter to only 'observation' packets authored since 2015/12/01.
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.authored_since:datetime(2015,12,1,tzinfo=pytz.UTC),
}

In [ ]:
#The `map_` endpoints return dictionaries summarising the matching result-set:
apiv1.map_stream_count(filters=my_filters)

In [ ]:
#Get the IVORN of the most recently authored SWIFT GRB alert:
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.ivorn_contains:'BAT_GRB', #case sensitive
}

swift_bat_grb_list = apiv1.list_ivorn(
    filters=my_filters,
    order=OrderValues.author_datetime_desc,
    n_max=1
)
swift_bat_grb_ivorn = swift_bat_grb_list[0]
swift_bat_grb_ivorn

In [ ]:
# Retrieve a 'synopsis' (nested dictionary) of the VOEvent packet,
grb_nested_dict = apiv1.packet_synopsis(swift_bat_grb_ivorn)
# And convert it to a user-friendly class-object:
grb_synopsis = Synopsis(grb_nested_dict)
print(grb_synopsis.ivorn)
print(grb_synopsis.author_ivorn)
print(grb_synopsis.author_datetime)
print(grb_synopsis.sky_events)

In [ ]:
#Note that the associated event position has been parsed into
# astropy.coordinates.SkyCoord format:
sky_event=grb_synopsis.sky_events[0]
sky_event.position

In [ ]:
#We can search for VOEvents with nearby co-ordinates:
from astropy.coordinates import Angle
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.cone: (sky_event.position, Angle(1.0, unit='deg'))
    }
apiv1.map_stream_count(my_filters)

In [ ]:
#Or retrieve the original XML for the packet:
raw_xml = apiv1.packet_xml(swift_bat_grb_ivorn)
print(raw_xml[:72])
print(' ... ')
